In [1699]:
#the xlrd library is necessary for read xls file  
#%pip install xlrd 

import pandas as pd
import numpy as np
import regex as re
from datetime import datetime
from time import time
from math import ceil
from datetime import timedelta
import unicodedata  
import os


In [1700]:
start_time = time()
#documentating information about the excel files and sheets inside of it.

#enter in the Data_Source folder to read the files
%cd Data_Source
#listing the files on the directory
files = os.listdir()

#getting all the files with xlsx or xls format
xlsx_files = [x for x in files if x.split('.')[-1]=='xlsx' or  x.split('.')[-1]=='xls']

def get_sheets_names(x):
    xlsx = pd.ExcelFile(x)
    return xlsx.sheet_names  # see all sheet names

#putting all the sheets names inside a list
sheets_names = [get_sheets_names(excel) for excel in xlsx_files]

#zipping the lists togueter for execute as program
lst_src = list(zip(xlsx_files, sheets_names))

#building a dataframe with information about the excel files
df_xlsx = pd.DataFrame(lst_src, columns=['xlsx_file','sheets'])

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time

for source in lst_src: print(f'EXCEL: {source[0]} >>> SHEETS: {source[1]}')

/home/roberto/git_trash/projeto_controle_financeiro_gesac-2/Data_Source
Time: This code took 2 seconds to run.
EXCEL: Controle de Empenhos e NC 2022.xlsx >>> SHEETS: ['ControleGeral', 'SintéticoGeral', 'SintéticoNC', 'AnalíticoNC', 'ÍndiceCorreçãoNC']
EXCEL: Controle de NC 2022.xlsx >>> SHEETS: ['SintéticoNC', 'AnalíticoNC', 'ÍndiceCorreção']
EXCEL: Emendas2021_resumo.xlsx >>> SHEETS: ['Planilha1']
EXCEL: Proponentes.xlsx >>> SHEETS: ['Proponentes']
EXCEL: Relatorio_SISGESAC 1-02-2022.xls >>> SHEETS: ['Extração']


In [1701]:
start_time = time()

df_nm =[
('df_an', 'AnalíticoNC'),
('df_sn', 'SintéticoNC'), 
('df_ic', 'ÍndiceCorreção'),
('df_pr', 'Proponentes'),
('df_cg', 'ControleGeral'),
('df_em', 'Planilha1'),
('df_sg', 'Extração')] 

df_sheets = pd.DataFrame(df_nm, columns = ['df_name', 'sheet_name'])
df_sheets['xlsx_file']= ''

for i1, sheets in enumerate(df_xlsx['sheets']):
    for i2, sheet in enumerate(df_sheets['sheet_name']):
        if sheet in sheets:
            df_sheets['xlsx_file'][i2] = df_xlsx['xlsx_file'][i1] 


def parameters(df):
    excel = df_sheets.loc[df_sheets.df_name == df]['xlsx_file'].values[0]
    sheet = df_sheets.loc[df_sheets.df_name == df]['sheet_name'].values[0]
    return excel, sheet


xlsx, sheet = parameters('df_an')
df_an = pd.read_excel(xlsx,sheet_name=sheet,
usecols="B:J", skiprows=1,parse_dates=True)

xlsx, sheet = parameters('df_sn')
df_sn = pd.read_excel(xlsx, sheet_name=sheet,
usecols="B:J", skiprows=3,parse_dates=True)

xlsx, sheet = parameters('df_ic')
df_ic = pd.read_excel(xlsx, sheet_name=sheet,
skiprows=1,usecols="B:E")

xlsx, sheet = parameters('df_pr')
df_pr = pd.read_excel(xlsx)

xlsx, sheet = parameters('df_cg')
df_cg = pd.read_excel(xlsx, sheet_name=sheet
, skiprows=12,usecols="c:w")

xlsx, sheet = parameters('df_em')
df_em= pd.read_excel(xlsx)

xlsx, sheet = parameters('df_sg')
df_sg= pd.read_excel(xlsx)
#----------------------------------------------------------
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time
#Getting out of the Data_Source folder after having read the files
%cd ..

/home/roberto/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
/home/roberto/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Slicer List extension is not supported and will be removed
  warn(msg)


Time: This code took 4 seconds to run.
/home/roberto/git_trash/projeto_controle_financeiro_gesac-2


In [1702]:
#In general the excel files with just one sheet don't have a good name, so the name was changed to be similar to the name of the excel file

for i, sheet in enumerate(df_xlsx['sheets']):
    if len(sheet)==1:
        new_name = df_xlsx['xlsx_file'][i]
        re_pattern = '(.xlsx?|\d|-|\s)'
        new_name = re.sub(re_pattern,'',new_name)
        df_sheets.sheet_name.loc[df_sheets.sheet_name == sheet[0]] = new_name
    else: pass

for index_1, xlsx_1 in enumerate(df_xlsx['xlsx_file']):
    for index_2, xlsx_2 in enumerate(df_sheets['xlsx_file']):
        if xlsx_1 == xlsx_2 and len(df_xlsx['sheets'][index_1])==1:
            df_xlsx['sheets'][index_1] = df_sheets['sheet_name'][index_2]
        else: pass

df_xlsx

,xlsx_file,sheets
0,Controle de Empenhos e NC 2022.xlsx,"[ControleGeral, SintéticoGeral, SintéticoNC, A..."
1,Controle de NC 2022.xlsx,"[SintéticoNC, AnalíticoNC, ÍndiceCorreção]"
2,Emendas2021_resumo.xlsx,Emendas_resumo
3,Proponentes.xlsx,Proponentes
4,Relatorio_SISGESAC 1-02-2022.xls,Relatorio_SISGESAC


In [1703]:
df_sheets

,df_name,sheet_name,xlsx_file
0,df_an,AnalíticoNC,Controle de NC 2022.xlsx
1,df_sn,SintéticoNC,Controle de NC 2022.xlsx
2,df_ic,ÍndiceCorreção,Controle de NC 2022.xlsx
3,df_pr,Proponentes,Proponentes.xlsx
4,df_cg,ControleGeral,Controle de Empenhos e NC 2022.xlsx
5,df_em,Emendas_resumo,Emendas2021_resumo.xlsx
6,df_sg,Relatorio_SISGESAC,Relatorio_SISGESAC 1-02-2022.xls


In [1704]:
start_time = time()
# criando função 'regexing' p/ subst. nomes abreviados de forma a normalizar as strings


lst_re = [
('PRF ','POLICIA RODOVIARIA FEDERAL '),
('MCOM ', 'MINISTERIO DAS COMUNICACOES '),
('BANCADA ','BANCADA-'),
(r'DA BANCADA-?',''),

(r'-D\w ','-'),
(r'\s(PARA|AJUDAR|DURANTE|A PANDEMIA DE COVID-19|DE\.)\s',''),
(r'\s(PROJETO|DA BANCADA|WIFI-FI NA|PRACA|ALTERANDO)\s',''),
(r'^PONTO\s(.*)DIA$',''),
(r'(\n)',''),
(r'SEN\.?(DOR\w?)?',''),
(r'(\s?DEPA?\.|\s?DEPUTAD\w?)\s?\s?(FEDERAL)?\s?',''),
(r'MIN/.?\s','MINISTERIO '),
(r'SOLICITA(CAO|d\w|N?DO)\s?(D\w|PEL\w|PARA\sO|PARA\sA)?',''),
(r'\w?\s?PEDIDO\s(FEIT\w|D\w)?\s?(PEL\w)?',''),
(r'-\s\w*',''),
(r'\(.*\)','')]
 

def regexing(value):
   for string in lst_re:
        value = re.sub(string[0],string[1],value)
   return value

lst_proponente = []

for index, prop in enumerate(df_pr['prop_pk1']):
    if df_pr['orgao'][index] == "senado":
        lst_proponente.append('SEN. ' + prop)
    elif  df_pr['orgao'][index] == "camara":
        lst_proponente.append('DEP. ' + prop)
    else: 
        lst_proponente.append(prop)

df_pr['prop_pk2'] = lst_proponente

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time
#Time: This ckode took 1 seconds to run.

Time: This code took 1 seconds to run.


In [1705]:
start_time = time()

def strip_accents(cln):
   cln = str(cln)
   return ''.join(c for c in unicodedata.normalize('NFD', cln)if unicodedata.category(c) != 'Mn')

def clean_column_name(cln):
    return strip_accents(cln.strip('[]()').lower().replace(' ','_'))

def rename_df(df):
    new_cln = [clean_column_name(cln) for cln in df.columns.values]
    old_cln = [cln for cln in df.columns.values]
    dct = dict(zip(old_cln, new_cln))
    df.rename(columns=dct,inplace=True)
    lst_df_clns.append(new_cln)
    return df

#creating a list with all dfs from the data dictionary
dfs = [ x for x in df_sheets.df_name]
cmd = str(dfs).replace("'","") 
exec('dfs = {}'.format(cmd))

lst_df_clns = []
for df in dfs:
    rename_df(df)


df_sheets['columns'] = lst_df_clns 

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time = end_time-start_time

df_sheets

Time: This code took 1 seconds to run.


,df_name,sheet_name,xlsx_file,columns
0,df_an,AnalíticoNC,Controle de NC 2022.xlsx,"[proponente, nota_de_credito, programa, mes, v..."
1,df_sn,SintéticoNC,Controle de NC 2022.xlsx,"[nc, proponente, programa, empenho, valor_inic..."
2,df_ic,ÍndiceCorreção,Controle de NC 2022.xlsx,"[mes, indice_de_correcao, taxa_selic, observacao]"
3,df_pr,Proponentes,Proponentes.xlsx,"[prop_pk1, uf, partido, orgao, prop_1, prop_2,..."
4,df_cg,ControleGeral,Controle de Empenhos e NC 2022.xlsx,"[processo, proponente, contrato, nota_de_empen..."
5,df_em,Emendas_resumo,Emendas2021_resumo.xlsx,"[processo, emenda, valor, pontos_livre_(18_mes..."
6,df_sg,Relatorio_SISGESAC,Relatorio_SISGESAC 1-02-2022.xls,"[cdgesac, nrpid, situacao, sgunidadefederacao,..."


In [1706]:
start_time = time()

df_sg['bancada'] = df_sg['nosolicitante'] 
df_sg['proponente'] = df_sg['dsjustificativa'] 
df_em['proponente'] = df_em['emenda']

#fazendo o loop sobre a lista acima para aplicar as funções que normalizam a coluna com as chaves dos proponentes

lst_df = [df_sn, df_an, df_cg, df_em, df_sg]
for df in lst_df: 
    df['proponente'] = df['proponente'].apply(lambda x:strip_accents(x).upper()).apply(lambda x:regexing(x))
    df['proponente'] = df['proponente'].apply(lambda x:str(x).strip())


lst_pr = (df_pr['prop_pk1'], df_pr['prop_1'],df_pr['prop_2'], df_pr['prop_3'],df_pr['prop_4'])

for cln in lst_pr:
    cln = cln.apply(lambda x:strip_accents(x).upper())
    
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 2 seconds to run.


In [1707]:
#criando lista dos DataFrames pelo qual o loop passara referente aos proponentes
dfs = (df_sn, df_an,df_cg, df_em, df_sg)

#criando as colunas das primary_keys nos 3 dataframes
for df in dfs:
    df['prop_pk1'] = ''
    df['prop_pk2'] = ''

O objetivo inicial do projeto foi limpar os dados relacionados ao os proponentes com a criação de uma coluna nova referente ao nome civil do proponente para que esses valores pudessem ser usados como chave entre diferentes tabelas que serao parametrizadas entre si.

In [1708]:
start_time = time()
#criando lista das colunas referente ao nome dos proponentes 
def filter_key(x):
    return (x == lst_pr[0])|(x==lst_pr[1])|(x==lst_pr[2])|(x==lst_pr[3])

for df in lst_df:
    lst_prop_pk1 = []
    lst_prop_pk2= []
    for index,value in enumerate(df.proponente):
        filt_finding_key =  filter_key(df.proponente.iloc[index])
        result1 = str(df_pr.loc[filt_finding_key].prop_pk1.values).strip("[']")
        result2 = str(df_pr.loc[filt_finding_key].prop_pk2.values).strip("[']")
        lst_prop_pk1.append(result1)
        lst_prop_pk2.append(result2)
    df['prop_pk1'] = lst_prop_pk1
    df['prop_pk2'] = lst_prop_pk2

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 21 seconds to run.


In [1709]:
start_time = time()
#cleaning analitic sintetic rows

lst = ['SUBTOTAL','TOTAL','NAN']

for value in lst:
    df_sn = df_sn.loc[df_sn['proponente'] != value ]  
    df_em = df_em.loc[df_em['proponente'] != value ]  
    
df_sn.fillna(0,inplace=True)
df_an.fillna(0,inplace=True)

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 1 seconds to run.


In [1710]:

df_em[['proponente','emenda','prop_pk1']].loc[df_em.prop_pk2 =='']

,proponente,emenda,prop_pk1
4,MISAEL VARELLA,Dep. Misael Varella,
5,GILBERTO ABRAMO,Dep. Gilberto Abramo (bancada MG),
6,EUCLYDES PETTER,Dep. Euclydes Pettersen,
7,AUREA CAROLINA,Dep. Áurea Carolina,
9,JULIO DELGADO,Dep. Júlio Delgado (bancada MG),
10,EROS BIONDINI,Dep. Eros Biondini,
11,BILAC PINTO,Dep. Bilac Pinto,
12,EDUARDO BARBOSA,Dep. Eduardo Barbosa,
13,MAURO LOPES,Dep. Mauro Lopes,
14,ANDRE JANONES,Dep. André Janones,


In [1711]:
df_sg

len(df_sg[['proponente','prop_pk1','bancada']].loc[df_sg.prop_pk1 != ''].prop_pk1.unique())
len(df_sg[['proponente','prop_pk1','bancada']].loc[df_sg.prop_pk1 == ''].proponente.unique())


249

array(['MIGRACAO', 'MCOM', 'WI-FI NA MINISTRO DAS COMUNICACOES',
       'MINISTRO DAS COMUNICACOES', 'CELSO SABINO', 'GIL CUTRIM.',
       'CLEBER VERDE', 'CLEBER VERDE.', 'OCLEBER VERDE',
       'MINISTERIO DA JUSTICA', 'MINISTERIO DA JUSTICA.',
       'MARIANA CARVALHO.', 'MARIANA CARVALHO', 'CENSIPAM.', 'CENSIPAN',
       'MINISTERIO DAS COMUNICACOES', 'MINISTERIO DAS COMUNICACOES.',
       'ADOR CIRO NIGUEIRA.', 'ADOR CIRO NOGUEIRA.', 'PERPETUA ALMEIDA.',
       'CELSO SABINO.', 'VICENTINHO JUNIOR', 'CIDADANIA',
       'FEITO PELA CIDADANIA', 'JOAO ROMA',
       'PROJETO WI-FI NA PRACAATENDIMENTO DA COMUNIDADE LOCAL.',
       'EDUARDO BRAIDE', 'JOSE MEDEIROS', 'JOSE MEDEIROS.',
       'EDUARDO BRAIDE.', 'PRF-AM', 'WLADIMIR GAROTINHO',
       'MINISTERIO DA SAUDE', 'WI-FI NA PRACA', 'MINICOM',
       'ADORA ELIZIANE GAMA', 'PARLAMENTAR', 'WI-FI NA', 'ASP', 'MINICON',
       'JOAQUIM PASSARINHO', 'SESAI', 'GIL CUTRIM', 'JOAO DANIEL',
       'SESAI.', 'MMA', 'INTERNET', 'EVAIR DE MELO

In [1712]:
start_time = time()
dfs = [df_an, df_ic]

for index,df in enumerate(dfs): 
    meses = [mes for mes in df.mes.unique()]
    n_mes = ['01/2021','02/2021','03/2021','04/2021','05/2021','06/2021','07/2021','08/2021','09/2021','10/2021','11/2021','12/2021']
    meses = list(zip(meses, n_mes))

    for index,value in enumerate(meses):
        df['mes'].replace(meses[index][0],meses[index][1],inplace=True)


    df['mes']= pd.to_datetime(df['mes'])
    if (df.mes.dtype == '<M8[ns]') == True:
        print(f'The loop is ok!')
        
    else: print('The loops is broke!')
    
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

The loop is ok!
The loop is ok!
Time: This code took 1 seconds to run.


In [1713]:
#checando se tem dfs sem prop_pk 
start_time = time()

dfs  = [df_an, df_sn,df_em, df_sg]
lst_no_cv_name = []
for df in dfs:
    for index, value in enumerate(df.loc[df.prop_pk1==''].proponente.values):
        if value not in lst_no_cv_name:
            lst_no_cv_name.append(df.loc[df.prop_pk1==''].proponente.values[index])
        else:pass

print('DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:')
df_pr_nokey = pd.DataFrame()
df_pr_nokey['prop_nokey']= lst_no_cv_name
df_pr_nokey['prop_pk1']= ''
df_pr_nokey['orgao']=''
df_pr_nokey['uf']=''
df_pr_nokey['partido']=''

save = True
if save == True:
    %mkdir trash
    df_pr_nokey.to_excel('trash/props_with_no_key.xlsx')
else:pass

end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

#df_em.loc[df_em.prop_pk1!='']['prop_pk1']
df_pr_nokey.prop_nokey.unique()


DF COM OS PROPONENTES QUE NÃO APRESENTARAM CORRESPONDÊNCIA COM A COLUNA DO NOME CIVIL:
mkdir: cannot create directory ‘trash’: File exists
Time: This code took 5 seconds to run.


array(['BANCADA-ALAGOAS', 'BANCADA-MINAS GERAIS', 'BANCADA-SAO PAULO',
       'BANCADA-ESPIRITO SANTO', 'BANCADA-MARANHAO', 'BANCADA-PARA',
       'ADOR CONFUCIO MOURA', 'MISAEL VARELLA', 'GILBERTO ABRAMO',
       'EUCLYDES PETTER', 'AUREA CAROLINA', 'JULIO DELGADO',
       'EROS BIONDINI', 'BILAC PINTO', 'EDUARDO BARBOSA', 'MAURO LOPES',
       'ANDRE JANONES', 'PAULO GUEDES', 'SUBTENENTE GONZAGA', 'IGOR TIMO',
       'PATRUS ANANIAS', 'FREDERICO', 'MARIO HERINGER',
       'LEONARDO MONTEIRO', 'CABO JUNIO AMARAL BANCADA', 'LUCAS GONZALEZ',
       'VILSON DA FETAEMG', 'NEWTON CARDOSO', 'ZE SILVA',
       'LINCOLN PORTELA', 'AECIO NEVES', 'CHARLES EVANGELISTA',
       'ZE VITOR', 'DELEGADO MARCELO FREITAS', 'STEFANO AGUIAR',
       'WELITON PRADO', 'FRANCO CARTAFINA', 'PAULO ABI ACKEL',
       'MARCELO ARO', 'ROGERIO CORREIA', 'PADRE JOAO', 'LAURIETE',
       'AMARO NETO', 'DA VITORIA', 'GASTAO VIEIRA',
       'CABO JUNIO AMARAL  INDIVIDUAL', 'LUIS TIBE  INDIVIDUAL',
       'MIGRACAO', 

In [1714]:
df_sheets

,df_name,sheet_name,xlsx_file,columns
0,df_an,AnalíticoNC,Controle de NC 2022.xlsx,"[proponente, nota_de_credito, programa, mes, v..."
1,df_sn,SintéticoNC,Controle de NC 2022.xlsx,"[nc, proponente, programa, empenho, valor_inic..."
2,df_ic,ÍndiceCorreção,Controle de NC 2022.xlsx,"[mes, indice_de_correcao, taxa_selic, observacao]"
3,df_pr,Proponentes,Proponentes.xlsx,"[prop_pk1, uf, partido, orgao, prop_1, prop_2,..."
4,df_cg,ControleGeral,Controle de Empenhos e NC 2022.xlsx,"[processo, proponente, contrato, nota_de_empen..."
5,df_em,Emendas_resumo,Emendas2021_resumo.xlsx,"[processo, emenda, valor, pontos_livre_(18_mes..."
6,df_sg,Relatorio_SISGESAC,Relatorio_SISGESAC 1-02-2022.xls,"[cdgesac, nrpid, situacao, sgunidadefederacao,..."


In [1715]:
start_time = time()

#installing the python library to save the data an excel with all the sheets
%rm -r PowerBI_Dataset
%mkdir PowerBI_Dataset
%mkdir PowerBI_Dataset/csv/
%mkdir PowerBI_Dataset/xlsx

path = 'PowerBI_Dataset' 
#putting all dfs inside the excel
for i, v in enumerate(df_sheets['df_name']):
  command1 = (f"{df_sheets['df_name'][i]}.to_csv('{path}/csv/{v}.csv')")
  command2 = (f"{df_sheets['df_name'][i]}.to_excel('{path}/xlsx/{v}.xlsx')")
  exec(command1)
  exec(command2)
  
end_time = time(); print(f'Time: This code took {ceil(end_time-start_time)} seconds to run.'); total_time += end_time-start_time

Time: This code took 13 seconds to run.


In [1716]:
%rm -r Data_Dictionary
%mkdir Data_Dictionary
%mkdir Data_Dictionary/csv/
%mkdir Data_Dictionary/xlsx/
%mkdir Data_Dictionary/csv/Columns/
%mkdir Data_Dictionary/xlsx/Columns/

path = 'Data_Dictionary'

for df in ('df_sheets', 'df_xlsx'):
    exec(f"{df}.to_csv('{path}/csv/{df}.csv')")
    exec(f"{df}.to_excel('{path}/xlsx/{df}.xlsx')")


#df_nm = 'df_xlsx'
#df = exec(f'{df_nm}') 
#exec(f"{df}.to_csv('{path}/csv/{df_nm}.csv')")
#exec(f"{df]}.to_excel('{path}/xlsx/{df_nm}.xlsx')")

path2 = 'Columns'

for i, df  in enumerate(df_sheets['df_name']):
       exec(f'df = {df}')
       lst_cl = []
       for cl in df.columns: lst_cl.append(cl)
       dct = {'columns':lst_cl}
       exec(f"{df_sheets['df_name'][i]}_columns = pd.DataFrame(dct)")
       exec(f"{df_sheets['df_name'][i]}_columns['descricao'] = '' ")
       exec(f"{df_sheets['df_name'][i]}_columns.to_csv('{path}/csv/{path2}/{df_sheets['sheet_name'][i]}_columns.csv')")
       exec(f"{df_sheets['df_name'][i]}_columns.to_excel('{path}/xlsx/{path2}/{df_sheets['sheet_name'][i]}_columns.xlsx')")

df_sheets


,df_name,sheet_name,xlsx_file,columns
0,df_an,AnalíticoNC,Controle de NC 2022.xlsx,"[proponente, nota_de_credito, programa, mes, v..."
1,df_sn,SintéticoNC,Controle de NC 2022.xlsx,"[nc, proponente, programa, empenho, valor_inic..."
2,df_ic,ÍndiceCorreção,Controle de NC 2022.xlsx,"[mes, indice_de_correcao, taxa_selic, observacao]"
3,df_pr,Proponentes,Proponentes.xlsx,"[prop_pk1, uf, partido, orgao, prop_1, prop_2,..."
4,df_cg,ControleGeral,Controle de Empenhos e NC 2022.xlsx,"[processo, proponente, contrato, nota_de_empen..."
5,df_em,Emendas_resumo,Emendas2021_resumo.xlsx,"[processo, emenda, valor, pontos_livre_(18_mes..."
6,df_sg,Relatorio_SISGESAC,Relatorio_SISGESAC 1-02-2022.xls,"[cdgesac, nrpid, situacao, sgunidadefederacao,..."


In [1717]:
final_time= str(timedelta(seconds= total_time)); print(f'Time: The total time to run all the notebook was {final_time}.')

Time: The total time to run all the notebook was 0:00:39.733881.
